In [1]:
import nltk
import spacy
from spacy import displacy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime
import re
from collections import Counter
from itertools import chain
from nltk.corpus import stopwords

%matplotlib inline
pd.options.display.max_columns = None

In [2]:
data = pd.read_csv('/home/haomiao/work/target0/data/initial_cleaned_text_Airsweb.csv')

/home/haomiao/python_venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21,22,23,24,39,42,43,62,63,64,67,71,72,75,76,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Amey Plc (ALL),System AI ID,Event type,Event subtype,Who has reported this incident?,Are you an operative/worker or supervisor?,About the affected person,Type of person,UK Rail related?,Which Company employs them if not Amey?,Date/time of incident,Date/Time Reported,Business Unit,Sector,Contract,Site / account,Where was it?,Location,Specific location?,Do you work in the Railway part of Amey?,Contact telephone number,Territory,Asset,Project number,Programme Manager,Were you working alone?,Describe the task being done at the time,Highway Work Breakdown Structure / Service,Was affected person working alone?,Describe the incident:,What did you see?,What could have happened?,What were you able to do about it?,Were you at work or travelling?,What were the lighting conditions?,Was this a Railway Near Miss involving a train/on-track machine/on-track plant?,Who might have caused it?,Immediate remedial actions taken,Part of body that has been injured?,Injury type?,Illness Type,Names of any witnesses,NSC 24:7 reference number,Test,"If positive, for what?",Accident type,Causal analysis,"Investigation findings, if any",NDS 24:7 reference number for Railway incidents,Any plant or tools involved?,"Degree of Potential Harm (1=low, 3=high)","Seriousness? (1=low, 3=high)",Is further follow-up required?,Description of follow-up,Has follow-up been completed?,Potential harm from,Highway Work Breakdown Structure / Service408,Root cause for Railway Incident,Potential environmental impact,Learning Event,Learning Event details,Deemed significant by Railway Professional Head?,Risks posed by incident,Factors to be considered by manager,Current status,Share with Industry?,Report complete and closed,Experience in company,Event subtype53,Legally reportable locally by Amey/subcontractor?,Include in Amey+subcontractor statistics?,Expected lost days,Actual lost days,Expected restricted days,Actual restricted days,Start of sick leave,Date returned to work on full duties,Name of Lead Investigator,Position,Investigation sign off date,Incident closed,clean_incident_description,clean_remedial,clean_what_did_you_see,clean_what_did_you_do,clean_what_could_have_happen,clean_description_follow_up,clean_learning_event,clean_investigation_findings,clean_factors_tobe_considered,clean_risks_posed
0,Amey Plc (ALL),107541,Close Call or Near Miss,I - Close Call,"Gibson, Raymond",Operative,",",1 - Amey Employee,NaN,NaN,2015-01-02T08:00:00Z,2015-01-02T08:24:00Z,Consulting and Rail,Rail,Rail CEFA,Rail - CEFA Scotland,02 Site,NaN,"Haymarket ,Edinburgh.",N,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,N,NaN,Raymond did a last check before leaving the jo...,Derail a train worse case .,Cleared away barrette off the rail line and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1 - Realistic potential for NLTI,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,N,Y,NaN,I - Close Call,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,Near Miss Auto-Complete,Near Miss Auto-Complete,NaN,Y,NaN,NaN,raymond did a last check before leaving the jo...,cleared away barrette off the rail line and us...,derail a train worse case,NaN,NaN,NaN,NaN,NaN
1,Amey Plc (ALL),107543,Close Call or Near Miss,I - Close Call,"Turner , Justin",Operative,",",3 - Subcontractor/Contractor,NaN,NaN,2015-01-02T08:30:00Z,2015-01-02T08:43:00Z,Consulting and Rail,Rail,Rail CEFA,Consulting - CEFA Kent & Sussex,02 Site,NaN,Lift in the south East Hub.,N,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,N,NaN,Justin is reporting that the lift has dropped ...,NaN,Justin has reported this to his manager Andy D...,At work,Artificial,N,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 Office,1.0,2 - Realistic potential for LTI to a single pe...,Y,Lift contractor called by office manager to in...,Y,A2 - Contact/near contact with moving faulty/d...,NaN,NaN,12 None,No,NaN,N,NaN,NaN,NaN,NaN,Y,NaN,I - Close Call,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,Near Miss Auto-Complete,Near Miss Auto-Complete,NaN,Y,NaN,NaN,justin is reporting that the lift has dropped ...,justin 

In [3]:
data['clean_remedial'].dropna(inplace= True)
remedial = data['clean_remedial'].tolist()

raw_remedial = data['Immediate remedial actions taken'].tolist()

#remove nan
cleaned_raw_remedial = [x for x in raw_remedial if str(x) != 'nan']

In [126]:
#tokenise via nltk
tagged_texts = nltk.pos_tag_sents(map(nltk.word_tokenize, remedial))
tagged_texts_list = list(chain(*tagged_texts))

In [8]:
pd.Series(Counter(i for i in tagged_texts_list)).sort_values(ascending = False)

to          TO      1638
the         DT      1556
and         CC      1181
a           DT       476
was         VBD      473
of          IN       408
on          IN       377
in          IN       294
for         IN       259
is          VBZ      235
he          PRP      234
has         VBZ      227
be          VB       227
site        NN       218
will        MD       205
with        IN       203
at          IN       196
reported    VBN      194
manager     NN       193
by          IN       192
it          PRP      189
as          IN       171
his         PRP$     163
been        VBN      157
no          DT       156
reported    VBD      146
this        DT       139
taken       VBN      134
who         WP       127
were        VBD      116
                    ... 
goring      VBG        1
gordon      NN         1
            JJ         1
sanchez     NN         1
sands       NNS        1
grinding    VBG        1
grip        JJ         1
grit        NN         1
rush        NN         1


In [9]:
cleaned_raw_remedial

['Vehicle recovery is being arranged. Driver instructed by supervisor to go home to rest.',
 'Booked to see GP later today .',
 'The IP has gone to hospital A&E  unit to have injury assessed.\n\nNCS 24 ref 15237',
 'informed BT police',
 'Dave requested it was logged as a property damage.\nReported to Martin Sprague.',
 'Police have been informed and Traffic collision ref no : 11564',
 'The vehicle is still functional and he has notified his manager and sent a picture.',
 'Staff sent to the local hospital for check up, Fitter assessing the RRV at present and also NSC 24/7 informed of Incident:15306',
 'The local S&T team were brought in to assist. They inspected the points and detected signs of damage to the first stretcher bar consistent with a run thru or partial run thru. They were unable to repair the damage in the remaining time of the possession. A decision was made to handback with the fault on 83 points and clamped in the normal position to minimise disruption to the train serv

In [10]:
#POS tag on raw data via spacy
nlp = spacy.load('en')
doc_raw = map(nlp,cleaned_raw_remedial)
tokens_raw = [[(token, token.lemma_, token.tag_) for token in sent] for sent in doc_raw]

In [93]:
#tokenise via spacy

doc = map(nlp,remedial)
#tokens= [[(token2,token2.tag_) for token2 in token] for token in doc]
tokens= [[token2.lemma_ for token2 in sent] for sent in doc]

words = list(chain(*tokens))
pd.Series(Counter([str(w) for w in words])).sort_values(ascending = False)

#remove stop words

filtered_words = [str(word) for word in words if str(word) not in stopwords.words('english')]

# find most frequent words in remedial
pd.Series(Counter(w for w in filtered_words)).sort_values(ascending = False)
Counter(w for w in filtered_words).most_common(40)

[('-PRON-', 953),
 ('report', 422),
 ('call', 235),
 ('site', 218),
 ('manager', 202),
 ('take', 202),
 ('work', 190),
 ('inform', 164),
 ('amey', 152),
 ('police', 134),
 ('repair', 127),
 ('vehicle', 125),
 ('advise', 119),
 ('damage', 118),
 ('team', 117),
 ('contact', 116),
 ('hospital', 109),
 ('first', 107),
 ('incident', 102),
 ('spill', 99),
 ('back', 99),
 ('get', 98),
 ('go', 93),
 ('ref', 88),
 ('kit', 87),
 ('driver', 86),
 ('use', 85),
 ('ip', 84),
 ('line', 81),
 ('attend', 81),
 ('also', 80),
 ('stop', 80),
 ('check', 80),
 ('aid', 78),
 ('supervisor', 77),
 ('detail', 70),
 ('send', 70),
 ('make', 70),
 ('nr', 68),
 ('close', 68)]

In [36]:
#bigrams and trigrams
from nltk import ngrams

cleaned_raw_list = [nltk.word_tokenize(sent) for sent in cleaned_raw_remedial]
bigrams = [ngrams(sent, 2) for sent in cleaned_raw_list]
bigrams_list = list(chain(*bigrams))

pd.Series(Counter(gram for gram in bigrams_list)).sort_values(ascending = False)

to         the            196
of         the            144
Reported   to             117
to         be             112
has        been            83
in         the             75
and        the             75
reported   to              70
.          The             70
will       be              67
on         site            61
.          He              54
A          &               51
taken      to              49
&          E               49
on         the             48
S          &               46
first      aid             46
as         a               44
with       the             42
.          Reported        41
to         get             41
have       been            40
it         to              39
&          T               38
by         the             38
back       to              37
the        incident        35
for        the             33
and        reported        33
                         ... 
issues     ,                1
issued     with             1
          

In [48]:
#trigrams
trigrams = [ngrams(sent, 3) for sent in cleaned_raw_list]
trigrams_list = list(chain(*trigrams))

words=[",",".",":"]
trigrams_list = [i for i in trigrams_list if i[0] not in words and i[1] not in words and i[2] not in words]
words1 = ["&"]
trigrams_list = [i for i in trigrams_list if i[0] not in words1 and i[2] not in words1]
pd.Series(Counter(gram for gram in trigrams_list)).sort_values(ascending = False)

A            &             E                     49
S            &             T                     38
reported     it            to                    19
GWE          Operations    Centre                17
and          reported      to                    16
reported     the           incident              16
a            close         call                  14
was          taken         to                    14
taken        to            hospital              13
to           go            to                    13
as           a             close                 13
to           line          manager               13
NSC          24/7          informed              13
to           see           if                    11
the          incident      to                    11
C            &             C                     11
reported     to            the                   11
complainant  to            compensate            10
sent         to            complainant           10
is          

In [144]:
trigrams_list

[('Vehicle', 'recovery', 'is'),
 ('recovery', 'is', 'being'),
 ('is', 'being', 'arranged'),
 ('Driver', 'instructed', 'by'),
 ('instructed', 'by', 'supervisor'),
 ('by', 'supervisor', 'to'),
 ('supervisor', 'to', 'go'),
 ('to', 'go', 'home'),
 ('go', 'home', 'to'),
 ('home', 'to', 'rest'),
 ('Booked', 'to', 'see'),
 ('to', 'see', 'GP'),
 ('see', 'GP', 'later'),
 ('GP', 'later', 'today'),
 ('The', 'IP', 'has'),
 ('IP', 'has', 'gone'),
 ('has', 'gone', 'to'),
 ('gone', 'to', 'hospital'),
 ('to', 'hospital', 'A'),
 ('A', '&', 'E'),
 ('E', 'unit', 'to'),
 ('unit', 'to', 'have'),
 ('to', 'have', 'injury'),
 ('have', 'injury', 'assessed'),
 ('NCS', '24', 'ref'),
 ('24', 'ref', '15237'),
 ('informed', 'BT', 'police'),
 ('Dave', 'requested', 'it'),
 ('requested', 'it', 'was'),
 ('it', 'was', 'logged'),
 ('was', 'logged', 'as'),
 ('logged', 'as', 'a'),
 ('as', 'a', 'property'),
 ('a', 'property', 'damage'),
 ('Reported', 'to', 'Martin'),
 ('to', 'Martin', 'Sprague'),
 ('Police', 'have', 'been')

In [92]:
#remove stop words
cleaned_raw_stop = []
for i in cleaned_raw_list:
    cleaned_raw_stop.append([word for word in i if word not in stopwords.words('english')])
trigrams_stop =[ngrams(sent, 3) for sent in cleaned_raw_stop]
trigrams_list_stop = list(chain(*trigrams_stop))

words=[",",".",":",";","(",")"]
trigrams_list_stop = [i for i in trigrams_list_stop if i[0] not in words and i[1] not in words and i[2] not in words]
words1 = ["&"]
trigrams_list_stop = [i for i in trigrams_list_stop if i[0] not in words1 and i[2] not in words1]
pd.Series(Counter(gram for gram in trigrams_list_stop)).sort_values(ascending = False)

A            &             E                       49
S            &             T                       38
GWE          Operations    Centre                  17
NSC          24/7          informed                13
Reported     line          manager                 11
C            &             C                       11
Voucher      sent          complainant             10
complainant  compensate    amount                  10
sent         complainant   compensate              10
H            &             S                        8
Spill        kit           used                     8
S            &             C                        8
D            &             A                        8
Amey         Operations    Centre                   7
             Ops           Centre                   7
             GWE           Operations               7
Reported     close         call                     7
address      added         mailing                  6
first        aid           k

In [70]:
pd.Series(Counter(gram for gram in trigrams_stop)).sort_values(ascending = False)

A            &            E              49
S            &            T              38
GWE          Operations   Centre         17
NSC          24/7         informed       13
Reported     line         manager        11
C            &            C              11
NR           C            &              10
Voucher      sent         complainant    10
close        call         .              10
complainant  compensate   amount         10
sent         complainant  compensate     10
.            First        aid            10
compensate   amount       .               9
.            S            &               9
             The          driver          9
             Spill        kits            9
             Voucher      sent            9
H            &            S               8
amount       .            Voucher         8
D            &            A               8
S            &            C               8
Spill        kit          used            8
Reported     close        call  

In [101]:
#cleaned_list
cleaned_stop = []
for i in remedial:
    cleaned_stop.append([word for word in i.split() if word not in stopwords.words('english')])

#[str(word) for word in list(chain(*cleaned_raw_list)) if str(word) not in stopwords.words('english')]

In [109]:
str(t2[0][0])

'vehicle'

In [115]:
#word_tag count
nlp = spacy.load('en')
doc = map(nlp,remedial)

In [118]:
tokens2= [[(token2,token2.lemma_, token2.tag_, token2.is_stop) for token2 in token] for token in doc]

In [119]:
tokens2

[[(the, 'the', 'DT', True),
  (ip, 'ip', 'NN', False),
  (has, 'have', 'VBZ', True),
  (gone, 'go', 'VBN', False),
  (to, 'to', 'TO', True),
  (hospital, 'hospital', 'NN', False),
  (a, 'a', 'DT', True),
  (e, 'e', 'NN', False),
  (unit, 'unit', 'NN', False),
  (to, 'to', 'TO', True),
  (have, 'have', 'VB', True),
  (injury, 'injury', 'NN', False),
  (assessed, 'assess', 'VBN', False),
  (ncs, 'nc', 'NNS', False),
  (24, '24', 'CD', False),
  (ref, 'ref', 'NN', False),
  (15237, '15237', 'CD', False)],
 [(informed, 'inform', 'VBN', False),
  (bt, 'bt', 'VBZ', False),
  (police, 'police', 'NN', False)],
 [(dave, 'dave', 'NN', False),
  (requested, 'request', 'VBD', False),
  (it, '-PRON-', 'PRP', True),
  (was, 'be', 'VBD', True),
  (logged, 'log', 'VBN', False),
  (as, 'as', 'IN', True),
  (a, 'a', 'DT', True),
  (property, 'property', 'NN', False),
  (damage, 'damage', 'NN', False),
  (reported, 'report', 'VBN', False),
  (to, 'to', 'IN', True),
  (martin, 'martin', 'NN', False),
  (s

In [112]:
tokens= [[(token2,token2.tag_, token2.is_stop == False) for token2 in token] for token in doc]
t2=list(chain(*tokens))

pd.Series(Counter([str(i[0])+"_"+str(i[1]) for i in t2])).sort_values(ascending=False)
#have a look at most common words, bigram, n-gram (need stemming)

the_DT              1556
and_CC              1181
to_TO                898
to_IN                740
a_DT                 476
was_VBD              473
of_IN                408
on_IN                359
in_IN                268
for_IN               259
he_PRP               237
reported_VBN         236
is_VBZ               235
be_VB                227
has_VBZ              227
site_NN              206
will_MD              205
with_IN              203
manager_NN           193
at_IN                192
by_IN                192
it_PRP               189
as_IN                166
his_PRP$             163
been_VBN             157
no_DT                155
this_DT              139
taken_VBN            134
i_PRP                132
who_WP               127
                    ... 
strapped_VBN           1
flat_JJ                1
flagged_VBN            1
flag_NN                1
strategy_NN            1
stream_NN              1
stressing_VBG          1
five_CD                1
fm_NN                  1


In [145]:
verbs_incident = pd.read_csv('/home/haomiao/scratch/Target0/verbs_incident.csv')
nouns_incident = pd.read_csv('/home/haomiao/scratch/Target0/nouns_incident.csv')

In [146]:
verbs_incident

,Unnamed: 0,word,word_lemma,tag,count
0,5,left,leave,V,4210
1,10,fall,fall,V,3126
2,12,attached,attach,V,2892
3,18,sign,sign,V,2391
4,24,working,work,V,2217
5,28,missing,miss,V,1910
6,37,slip,slip,V,1533
7,43,following,follow,V,1384
8,46,causing,cause,V,1344
9,47,received,receive,V,1300


In [149]:
nouns_incident.loc[nouns_incident['word']=='hospital']

,Unnamed: 0,word,word_lemma,tag,count
793,1108,hospital,hospital,N,86
